# <center>  MODEL STRIDE - FC1 -   KS 7 </center>

# Imports


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import numpy as np
import os 
import glob
import sys
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
#from utils import TrainingMetrics
sys.path.insert(0, '/home/trz846/representation_learning/scripts/')
import utils
import time
import preprocess_data

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import pearsonr
from IPython.display import display, Markdown
import nglview as nv
import MDAnalysis as mda



_ColormakerRegistry()

# Import representation model

In [2]:
path = '/home/trz846/representation_learning/models/cnn_strided_ls500_1fc_seq63/str2_ks7/'

sys.path.insert(0, path)
from cnn_seq_stride_FC import ConvNet
kernel_size =  7
stride = 2
padding = 3
ks_pool = None
str_pool = None
pad_pool = None

# load architecture
model = ConvNet(kernel_size, stride, padding)


# load as checkpoint file 
trained_model_path = path +'checkpoint_model_epoch0_train_ds15.pt'
# define optimizer as needed to load chckpt-file, not used 
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

# load trained model weights into init model architectyre from file 
model, optimizer, epoch_start, train_ds, loss_list, acc_list = \
    utils.load_checkpoint(model, optimizer, filename=trained_model_path) 

model = model.eval()

# # transfer to gpu
# if  torch.cuda.is_available(): 
#     model =  utils.load_final_model(path+'model_final.pt')
#     model.cuda()
# else:
#     model =  torch.load(path+'model_final.pt', map_location=torch.device('cpu'))
#     model.eval()

# evaluation mode for model
# model = model.eval
# get encoder part only 
encoder = model.encoder



# Utils

In [3]:
aa_to_int = {
        'M':1,
        'R':2,
        'H':3,
        'K':4,
        'D':5,
        'E':6,
        'S':7,
        'T':8,
        'N':9,
        'Q':10,
        'C':11,
        'U':12, # Selenocystein. 
        'G':13,
        'P':14,
        'A':15,
        'V':16,
        'I':17,
        'F':18,
        'Y':19,
        'W':20,
        'L':21,
        'O':22, # Pyrrolysine
        'start':23,
        'stop':24 }


def fasta_2_input(fasta_file, max_l=498):
        '''convert ddg's fasta file to a sequence that the CNN models
        can read (like in preprocess_data.py )
        '''

        with open(fasta_file, 'r') as f:
            seq = ""
            for line in f:
                # add to protein seq
                if not line[0] == '>':
                    seq += line.replace("\n","")

            # convert aa to integers        
            int_seq = aa_seq_to_int(seq)
            ## pad seq with 0's ##
            int_seq = pad_seq(int_seq, max_l)

        return np.array(int_seq)

def aa_seq_to_int(seq):
    """
    Return the sequence converted to integers as a list 
    plus start(23) and stop(24) integers. From Unirep. 
    """
    return [23] + [aa_to_int[aa] for aa in seq] + [24]
    

def pad_seq(seq, max_l):
    '''
    Pads the integer sequence with 0's up to max_l+2 
    '''
    padded_seq = [0]*(max_l+2) # +2 as start and stop added to seq 
    padded_seq[:len(seq)] = seq

    return padded_seq

    

def get_repr_from_encoder(seq_input, encoder):
    '''get AE encoders latent space on specific protein ''' 
    with torch.no_grad():
        ### get WT representation ###
        seq = torch.tensor(seq_input,dtype=torch.long)
        if  torch.cuda.is_available(): 
            seq.cuda()
        # convert to one hot 
        seq = utils.to_one_hot(seq)

        # convert for channels to come first
        seq = torch.transpose(seq,0,1) #transpose dim 1,2 => channels=aa

        # add "batch" dimension=1 for model to work
        seq = seq [None, :, :]
        # get wt representation for seq from encoder model
        repr = encoder(seq)
        
        return repr
    
def get_pred(seq_input, model):
    '''get AE predictions on specific protein '''
    with torch.no_grad():
        ### get WT representation ###
        seq_input = torch.tensor(seq_input,dtype=torch.long)
        if  torch.cuda.is_available(): 
            seq_input=seq_input.cuda()
        # convert to one hot 
        seq = utils.to_one_hot(seq_input)


        # convert for channels to come first
        seq = torch.transpose(seq,0,1) #transpose dim 1,2 => channels=aa

        # add "batch" dimension=1 for model to work
        seq = seq [None, :, :]

        # get wt representation for seq from encoder model
        out = model(seq)
        
        # get accuracy 
        _, predicted = torch.max(out.data, dim=1)
        
        
        # get correct with pad
        seq_input = seq_input[None, :]
        correct_w_pad = (seq_input == predicted)
        
        # filter out padding (0)
        msk = seq_input != 0
        pred_msk = predicted[msk]
        target_msk = seq_input[msk]
        correct = (target_msk == pred_msk)
        len_protein = msk.sum().item()
        
        return correct, correct_w_pad,len_protein


def visual_prot(pdb,seq_input, model,cam_orient=None):
    ''' visualises protein in cell output, where red in AE incorrect pred'''
    # get MDanalysis stuff
    u = mda.Universe(pdb, pdb)
    protein = u.select_atoms("segid A")
    start_resid = protein.resids[0]
    end_resid = protein.resids[-1]

    # get nn model predictions for sequence
    correct, correct_w_pad, len_protein = get_pred(seq_input, model)
    correct_pos = correct

    # CA = protein.select_atoms("name CA")
    # print('pdb',len(CA), 'start', start_resid, 'end',end_resid)

    # add column for colouring the predictions
    u.add_TopologyAttr('tempfactors')

    # loop through all atoms in given resid and give same tempfactor according to prediction
    temp_factors = []
    for index,res in enumerate(u.atoms.residues):
        resid = res.resid
        pred = correct_pos[resid]
        for atom in u.atoms.residues[index].atoms:
            temp_factors.append(pred)
    

    # add b-factors
    u.atoms.tempfactors = temp_factors
    protein = u.select_atoms("segid A") # only have chain A 

    # load universe object in nglview
    t = nv.MDAnalysisTrajectory(protein)
    w = nv.NGLWidget(t)

    # define represention
    w.representations = [
        {"type": "cartoon", "params": {
            "sele": "protein", "color": "bfactor"
        }},
        {"type": "ball+stick", "params": {
            "sele": "hetero"
        }}
    ]
    # w.download_image(filename='str2_ks5.png', factor=4, antialias=True, trim=False, transparent=True)
    if cam_orient != None:
        try: 
             w._set_camera_orientation(cam_orient)
        except:
            pass
  
    w._remote_call("setSize", target="Widget", args=["400px", "400px"])

    # use gui 
    # w.display(gui=True, use_box=False)

    # show in dislay
    return  w
    
    

def repr_ddg(encoder, fn_fasta, fn_ddg, correct, lat_space=500):
    seq_input = fasta_2_input(fn_fasta, max_l=498)
    ddgs = pd.read_csv(fn_ddg, sep="\s+", header = None, 
                        names=['PDB', 'CHAIN', 'WT', 'RES', 'MUT', 'ddg'])

    # define output  matrix
    ls1 = lat_space # first :ls1 is wt representation
    ls2 = lat_space*2 # ls1:ls2 mut representation
    ls3 = lat_space*3 # ls2:ls3 = difference: wt repr - mut rep
    ls4 = lat_space*3 + 500 # ls2:ls3 is wt labels seq()
    ls5 = lat_space*3 + 2*500 # # ls2:ls3 is wt labels ()
    col_pos = ls5 # col in matrix that defines of mutation in seq
    col_wt  = ls5+1  # wt aa label 
    col_mut = ls5+2  # mut aa label 
    col_AE_pred = ls5+3 # column where AE pred are def
    col_ddg = ls5+4 # column where exp ddgs are determine
#    col_rf  = ls4+5 # column whree rand forrest are defined
    
    # initialise matrix, where rows = each mutation, and columns
    ddgs_repr = np.zeros((len(ddgs),ls5+5))

    ## get WT representation 
    wt_repr = get_repr_from_encoder(seq_input, encoder).cpu().numpy()

    ## add wt_repr to first 500 columns in ddgs_repr
    ddgs_repr[:,:ls1] = wt_repr

    ## add WT labels to first columns between 1000:1500
    ddgs_repr[:,ls3:ls4] = seq_input

    for indx,row in ddgs.iterrows():

        #create mut seq from wt seq
        mut_seq = np.array(seq_input, copy=True) 

        # get seq index of mutation
        mut_pos = row['RES'] # obs first (0) is startcodon so counts from 1

        # wt/ original aa in int 
        wt_aa_int = utils.get_triAA_to_int(row['WT'])  

        #  which aa its mutated to , convert mutation from tri code to int 
        mut_aa_int = utils.get_triAA_to_int(row['MUT'])

        # change seq to include mutation 
        mut_seq[mut_pos] = mut_aa_int   # change aa in position 
    
        ## get mutation repr from encoder
        mut_repr = get_repr_from_encoder(mut_seq, encoder).cpu().numpy()

        # add mut_repr to ddgs_repr
        ddgs_repr[indx,ls1:ls2] = mut_repr
        
        # difference in representations
        diff_repr = mut_repr - wt_repr
        
        # add diff_repr to ddg
        ddgs_repr[indx, ls2:ls3] =  diff_repr
        
        # add mut fasta labels
        ddgs_repr[indx, ls4:ls5] = mut_seq  

        # save mut position
        ddgs_repr[indx:, col_pos] = mut_pos

        # save WT and mut aa label
        ddgs_repr[indx:,col_wt]  =  wt_aa_int
        ddgs_repr[indx:,col_mut] =  mut_aa_int

        # track if autoencoder got pred of pos correct
        if correct[mut_pos]:
            ddgs_repr[indx:,col_AE_pred] = 1
        else :
            ddgs_repr[indx:,col_AE_pred] = 0

    # add ddg measured values to last column in ddgs_repr
    ddg_values =  np.array(ddgs['ddg'])
    ddgs_repr[:,col_ddg]=ddg_values

    ### Split in train and test set ###
    # keep track of splitting by indexing 
    shuffl_indx = np.arange(len(ddgs))
    np.random.shuffle(shuffl_indx)
    train_len = int(0.80 * len(ddgs))
    # random indexes
    train_indx = shuffl_indx [:train_len]
    test_indx = shuffl_indx [train_len:]

    # get train/test from random indx
    train_set = ddgs_repr[train_indx]
    test_set = ddgs_repr[test_indx]
    
    # jaja could have returned pandas df but so much trouble to do so  
    return  train_set, test_set


def train_rand_forest(train_set, test_set, lat_space=500,
                      params=None, n_trees = 120,  train_w_labels=False, train_w_labels_only = False,
                      train_repr_diff=False):
    
    ls1 = lat_space # first :ls1 is wt representation
    ls2 = lat_space*2 # ls1:ls2 mut representation
    ls3 = lat_space*3 # ls2:ls3 = difference: wt repr - mut rep
    ls4 = lat_space*3 + 500 # ls2:ls3 is wt labels seq()
    ls5 = lat_space*3 + 2*500 # # ls2:ls3 is wt labels ()

    # define x and y for training. 
    y_train =train_set[:,-1] 
    y_test = test_set[:,-1]

    # which features to include in training together with repr
    if train_w_labels:
        x_train = np.concatenate((train_set[:,:ls2], train_set[:,ls3:ls5]),axis=1)
        x_test  = np.concatenate((test_set[:,:ls2], test_set[:,ls3:ls5]),axis=1) 
    elif train_w_labels_only: # only use labels in training, no repr
        x_train = train_set[:,ls4:ls5] 
        x_test  = test_set[:,ls4:ls5] 
    elif train_repr_diff: 
        x_train = train_set[:,ls2:ls3] 
        x_test  = test_set[:,ls2:ls3] 
    else: # only use representation
        x_train = train_set[:,:ls2] 
        x_test  = test_set[:,:ls2] 
    
    # init model using input params 
    if params is not None: 
        n_trees = params['n_estimators']
        min_samples_split = params['min_samples_split']
        min_samples_leaf = params['min_samples_leaf']
        max_features = params['max_features']
        max_depth = params['max_depth']
        bootstrap = params['bootstrap']
        rfr = RandomForestRegressor(n_estimators = n_trees, 
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf,
                                    max_features=max_features,
                                    max_depth=max_depth,
                                    bootstrap=bootstrap, oob_score=bootstrap)

    else:     
        rfr = RandomForestRegressor(n_estimators = n_trees, oob_score=True)
    
    # train model     
    rfr.fit(x_train,y_train)
    # get model prediction  on test set
    y_pred = rfr.predict(x_test)
    # add pred to last column of original matrix
    test_set = np.column_stack((test_set, y_pred ))
 
    # loss/error function (MSE):
    rmsd = np.sqrt(np.mean((y_pred - y_test)**2 ))
    pearson = pearsonr(y_pred, y_test)
    
    # split in AE correctly predicted and incorrectly predicted
    AE_pred = test_set[:,-3] # col_AE_pred
    idx_cor = np.argwhere(AE_pred==1)
    idx_incor = np.argwhere(AE_pred==0)

    return rfr, test_set, idx_cor, idx_incor, rmsd, pearson

def CV_rand_forest(train_set, test_set, lat_space=500,
                    train_w_labels=False, train_w_labels_only = False,
                    train_repr_diff=False):
    
    ls1 = lat_space # first :ls1 is wt representation
    ls2 = lat_space*2 # ls1:ls2 mut representation
    ls3 = lat_space*3 # ls2:ls3 = difference: wt repr - mut rep
    ls4 = lat_space*3 + 500 # ls2:ls3 is wt labels seq()
    ls5 = lat_space*3 + 2*500 # # ls2:ls3 is wt labels ()

    # define x and y for training. 
    y_train =train_set[:,-1] 
    y_test = test_set[:,-1]

    # which features to include in training together with repr
    if train_w_labels:
        x_train = np.concatenate((train_set[:,:ls2], train_set[:,ls3:ls5]),axis=1)
        x_test  = np.concatenate((test_set[:,:ls2], test_set[:,ls3:ls5]),axis=1) 
    elif train_w_labels_only: # only use labels in training, no repr
        x_train = train_set[:,ls4:ls5] 
        x_test  = test_set[:,ls4:ls5] 
    elif train_repr_diff: 
        x_train = train_set[:,ls2:ls3] 
        x_test  = test_set[:,ls2:ls3] 
    else: # only use representation
        x_train = train_set[:,:ls2] 
        x_test  = test_set[:,:ls2] 
        

    # DEFINE FEATURES TO CV TEST 
    n_trees = np.arange(50,500,50)
    # Number of features to consider at every split
    max_feat = [ "auto",'sqrt','log2']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]

    random_grid = {'n_estimators': n_trees,
                   'max_features': max_feat,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    
    ## Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestRegressor()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 200, verbose=2, random_state=42, n_jobs = -1)
    #  Fit the random search model
    rf_random.fit(x_train, y_train)
    print ('BEST PARAMS:')
    print(rf_random.best_params_)
    best_params = rf_random.best_params_
    
    best_rf = rf_random.best_estimator_
    y_pred = best_rf.predict(x_test)

    #test_set = np.append(test_set, y_pred, axis=1 )
    test_set = np.column_stack((test_set, y_pred ))
 
    # loss/error function (MSE):
    rmsd = np.sqrt(np.mean((y_pred - y_test)**2 ))
    pearson = pearsonr(y_pred, y_test)
    
    # split in AE correctly predicted and incorrectly predicted
    AE_pred = test_set[:,-3] # col_AE_pred
    idx_cor = np.argwhere(AE_pred==1)
    idx_incor = np.argwhere(AE_pred==0)
 
    return best_rf,  test_set, idx_cor, idx_incor, rmsd, pearson, best_params
    


# ===========================================


In [ ]:
# fasta
fasta_file = '/home/trz846/representation_learning/data/ddgs/proteingmayo/raw/fasta/1PGA.fasta.txt'
seq_input = fasta_2_input(fasta_file, max_l=498)
pdb = "/home/trz846/representation_learning/data/ddgs/proteingmayo/processed/pdbs/1PGA_clean.pdb"

# get decoder prediction 
correct, correct_w_pad, len_protein = get_pred(seq_input, model)
acc =  correct.sum().item()/len_protein
acc_pad =  correct_w_pad.sum().item()/len(seq_input) 
#print ('Accuracy for model on protein: ', correct.sum().item()/len(seq_input))
Markdown('# G-PROTEIN MAYO \n\n ### Acc for model on protein: {:.3},  w padding: {:.3} '.format(acc, acc_pad))


In [ ]:
fasta_file = '/home/trz846/representation_learning/data/ddgs/proteingmayo/raw/fasta/1PGA.fasta.txt'
seq_input = fasta_2_input(fasta_file, max_l=498)
pdb = "/home/trz846/representation_learning/data/ddgs/proteingmayo/processed/pdbs/1PGA_clean.pdb"
# get decoder prediction 
correct, correct_w_pad, len_protein = get_pred(seq_input, model)
cam_orient = [-50.41122655731412, -19.45342212415893, 0.03521294890622212, 0, -13.891423120996551, 36.06632237414191, 37.76211579590341, 0, -13.618562184236861, 35.220922664931315, -38.64906216158845, 0, -25.09212589263916, -24.426129817962646, -25.942466735839844, 1]
w = visual_prot(pdb,seq_input, model,cam_orient)
w

### Random Forest CV hyperparameters 

In [ ]:
fn_fasta = '/home/trz846/representation_learning/data/ddgs/proteingmayo/raw/fasta/1PGA.fasta.txt'
fn_ddg = '/home/trz846/representation_learning/data/ddgs/proteingmayo/processed/ddgs/proteingmayo.txt'
seq_input = fasta_2_input(fn_fasta, max_l=498)
correct, correct_w_pad, len_protein = get_pred(seq_input, model)


#process data and get representation for seq, add in matrix with ddgs for training downstream models
train_set, test_set  =  repr_ddg(encoder, fn_fasta, fn_ddg, correct)

### train on difference in repres of mut andm wt
# # get best params from random crossval
base_rfr,  base_test_set, base_idx_cor, base_idx_incor, base_rmsd, base_pearson,best_params = CV_rand_forest(train_set, test_set, lat_space=500,
                    train_w_labels=False, train_w_labels_only = False,
                    train_repr_diff=True)

# # ## representation best params from random CV 
# train downstream RF  using best params
# best_params=
rfr, rf_test_set, idx_cor, idx_incor, rmsd, pearson = train_rand_forest(train_set, test_set, 
                      lat_space=500, params=best_params, train_w_labels=False, train_w_labels_only = False,
                        train_repr_diff=True)

print('nr of features',rfr.n_features_ )
print ('number of non_zeros in feature importance for repr; ', np.count_nonzero(rfr.feature_importances_))

# # ### PLOTTING ### 
n_trees = best_params['n_estimators']
fig, ax = plt.subplots(1, 2, sharey=False, figsize=(14, 7), constrained_layout=True) 
tst = 'Pearson: {:.2f} \nRMSD: {:.2f}'.format(pearson[0], rmsd)

ax[0].scatter(rf_test_set[:,-1][idx_cor], rf_test_set[:,-2][idx_cor], color='blue', label=tst)
ax[0].scatter(rf_test_set[:,-1][idx_incor], rf_test_set[:,-2][idx_incor], color='red', label='AE incorrect pred')
ax[0].legend(loc='upper left', fontsize=15)
ax[0].set_xlabel('Predicted ∆∆G', fontsize=17)
ax[0].set_ylabel(r'Experimental DMS  ', fontsize=17)
ax[0].set_title(('Trained on representation \nn_trees:{}, max_features:None').format(str(n_trees)), fontsize=20)
ax[0].tick_params(axis='both', labelsize=16)
ax[0].set_xlim(-2,4.1)
ax[0].set_ylim(-2,4.1)
ax[0].axhline(0, linewidth=2, color = 'k') 
ax[0].axvline(0, linewidth=2, color = 'k')
ax[0].plot(ax[0].get_xlim(), ax[0].get_ylim(), ls="--", c=".3")
# Feature importance
ax[1].bar(np.arange(rfr.n_features_),rfr.feature_importances_)
ax[1].set_xlabel('latent space representation ', fontsize=17)
ax[1].tick_params(axis='both', labelsize=16)
ax[1].set_title('feature importance', fontsize=20)

plt.savefig('plot_DMS_protein_g.pdf')





# ================================================

In [ ]:
# fasta
fasta_file = '/home/trz846/representation_learning/data/ddgs/dms_fowler/raw/fasta/1D5R.fasta.txt'
seq_input = fasta_2_input(fasta_file, max_l=498)
pdb = "/home/trz846/representation_learning/data/ddgs/dms_fowler/processed/pdbs/1D5R_clean.pdb"

# get decoder prediction 
correct, correct_w_pad, len_protein = get_pred(seq_input, model)
acc =  correct.sum().item()/len_protein
acc_pad =  correct_w_pad.sum().item()/len(seq_input) 
#print ('Accuracy for model on protein: ', correct.sum().item()/len(seq_input))
Markdown('# FOWLER - 1D5R \n\n ### Acc for model on protein: {:.3}, w padding: {:.3} '.format(acc, acc_pad))

In [ ]:
fasta_file = '/home/trz846/representation_learning/data/ddgs/dms_fowler/raw/fasta/1D5R.fasta.txt'
seq_input = fasta_2_input(fasta_file, max_l=498)
correct, correct_w_pad, len_protein = get_pred(seq_input, model)
pdb = "/home/trz846/representation_learning/data/ddgs/dms_fowler/processed/pdbs/1D5R_clean.pdb"
cam_orient = [96.09886478780022, 13.7298922893142, -56.73615663350132, 0, -11.692529797584232, 111.59712514641404, 7.201363156325524, 0, 57.190787359066526, -0.2548374001420871, 96.80724210755506, 0, -33.43182325363159, -83.1609878540039, -31.10585927963257, 1]
w = visual_prot(pdb,seq_input, model,cam_orient)
w

### Process fasta and  ddgs

In [ ]:
# ### baseline CV
fn_fasta = '/home/trz846/representation_learning/data/ddgs/dms_fowler/raw/fasta/1D5R.fasta.txt'
fn_ddg =  '/home/trz846/representation_learning/data/ddgs/dms_fowler/processed/ddgs/dms_fowler_1D5R.txt'
seq_input = fasta_2_input(fn_fasta, max_l=498)
correct, correct_w_pad, len_protein = get_pred(seq_input, model)


#process data and get representation for seq, add in matrix with ddgs for training downstream models
train_set, test_set  =  repr_ddg(encoder, fn_fasta, fn_ddg,correct)

# train on difference in repres of mut andm wt
# get best params from random crossval

base_rfr,  base_test_set, base_idx_cor, base_idx_incor, base_rmsd, base_pearson, best_params = CV_rand_forest(train_set, test_set, lat_space=504,
                    train_w_labels=False, train_w_labels_only = False,
                    train_repr_diff=True)

## representation best params from random CV 
# best_params=
rfr, rf_test_set, idx_cor, idx_incor, rmsd, pearson = train_rand_forest(train_set, test_set, 
                      lat_space=500, params=best_params, train_w_labels=False, train_w_labels_only = False,
                        train_repr_diff=True)
print('nr of features',rfr.n_features_ )
print ('non_zeros in feat import; ', np.count_nonzero(rfr.feature_importances_))


# # ### PLOTTING ### 
n_trees = best_params['n_estimators']
fig, ax = plt.subplots(1, 2, sharey=False, figsize=(14, 7), constrained_layout=True) 
tst = 'Pearson: {:.2f} \nRMSD: {:.2f}'.format(pearson[0], rmsd)

ax[0].scatter(rf_test_set[:,-1][idx_cor], rf_test_set[:,-2][idx_cor], color='blue', label=tst)
ax[0].scatter(rf_test_set[:,-1][idx_incor], rf_test_set[:,-2][idx_incor], color='red', label='AE incorrect pred')
ax[0].legend(loc='upper left', fontsize=15)
ax[0].set_xlabel('Predicted ∆∆G', fontsize=17)
ax[0].set_ylabel(r'Experimental DMS  ', fontsize=17)
ax[0].set_title(('Trained on representation \nn_trees:{}, max_features:None').format(str(n_trees)), fontsize=20)
ax[0].tick_params(axis='both', labelsize=16)
ax[0].set_xlim(-0.4,1.3)
ax[0].set_ylim(-0.4,1.3)
ax[0].axhline(0, linewidth=2, color = 'k') 
ax[0].axvline(0, linewidth=2, color = 'k')
ax[0].plot(ax[0].get_xlim(), ax[0].get_ylim(), ls="--", c=".3")
# Feature importance
ax[1].bar(np.arange(rfr.n_features_),rfr.feature_importances_)
ax[1].set_xlabel('latent space representation ', fontsize=17)
ax[1].tick_params(axis='both', labelsize=16)
ax[1].set_title('feature importance', fontsize=20)

plt.savefig('plot_DMS_1D5R.pdf')


# =============================================

In [ ]:
# fasta
fasta_file = '/home/trz846/representation_learning/data/ddgs/dms_fowler/raw/fasta/2H11.fasta.txt'
seq_input = fasta_2_input(fasta_file, max_l=498)
pdb = "/home/trz846/representation_learning/data/ddgs/dms_fowler/processed/pdbs/2H11_clean.pdb"

# get decoder prediction 
correct, correct_w_pad, len_protein = get_pred(seq_input, model)
acc =  correct.sum().item()/len_protein
acc_pad =  correct_w_pad.sum().item()/len(seq_input) 
#print ('Accuracy for model on protein: ', correct.sum().item()/len(seq_input))
Markdown('# FOWLER - 2H11 \n\n### Acc for model on protein: {:.3}, w padding: {:.3} '.format(acc, acc_pad))

In [ ]:
fasta_file = '/home/trz846/representation_learning/data/ddgs/dms_fowler/raw/fasta/2H11.fasta.txt'
seq_input = fasta_2_input(fasta_file, max_l=498)
pdb = "/home/trz846/representation_learning/data/ddgs/dms_fowler/processed/pdbs/2H11_clean.pdb"
# get decoder prediction 
correct, correct_w_pad, len_protein = get_pred(seq_input, model)
cam_orient = [79.94508876962709, -0.12655908503161215, 11.267336805141511, 0, 0.1289010513754849, 80.73518092406466, -0.007742323550699695, 0, -11.267310255844327, 0.025655849540216713, 79.94518857093267, 0, -63.86358833312988, -28.813554525375366, -85.85820388793945, 1]
w = visual_prot(pdb,seq_input, model,cam_orient)
w

### Random Forest

In [ ]:
# # files to use
fn_fasta = '/home/trz846/representation_learning/data/ddgs/dms_fowler/raw/fasta/2H11.fasta.txt'
fn_ddg =  '/home/trz846/representation_learning/data/ddgs/dms_fowler/processed/ddgs/dms_fowler_2H11.txt'
seq_input = fasta_2_input(fn_fasta, max_l=498)
correct, correct_w_pad, len_protein = get_pred(seq_input, model)
#process data and get representation for seq, add in matrix with ddgs for training downstream models
train_set, test_set  =  repr_ddg(encoder, fn_fasta, fn_ddg,correct)

# # train on difference in repres of mut andm wt
# get best params from random crossval
base_rfr,  base_test_set, base_idx_cor, base_idx_incor, base_rmsd, base_pearson,best_params = CV_rand_forest(train_set, test_set, lat_space=504,
                    train_w_labels=False, train_w_labels_only = False,
                    train_repr_diff=True)

# # train downstream RF  using best params
# best_params=
rfr, rf_test_set, idx_cor, idx_incor, rmsd, pearson = train_rand_forest(train_set, test_set, 
                      lat_space=500, params=best_params, train_w_labels=False, train_w_labels_only = False,
                        train_repr_diff=True)
print('nr of features',rfr.n_features_ )
print ('non_zeros in feat import; ', np.count_nonzero(rfr.feature_importances_))


# # ### PLOTTING ### 

n_trees = best_params['n_estimators']
fig, ax = plt.subplots(1, 2, sharey=False, figsize=(14, 7), constrained_layout=True) 
tst = 'Pearson: {:.2f} \nRMSD: {:.2f}'.format(pearson[0], rmsd)

ax[0].scatter(rf_test_set[:,-1][idx_cor], rf_test_set[:,-2][idx_cor], color='blue', label=tst)
ax[0].scatter(rf_test_set[:,-1][idx_incor], rf_test_set[:,-2][idx_incor], color='red', label='AE incorrect pred')
ax[0].legend(loc='upper left', fontsize=15)
ax[0].set_xlabel('Predicted ∆∆G', fontsize=17)
ax[0].set_ylabel(r'Experimental DMS  ', fontsize=17)
ax[0].set_title(('Trained on representation \nn_trees:{}, max_features:None').format(str(n_trees)), fontsize=20)
ax[0].tick_params(axis='both', labelsize=16)
ax[0].set_xlim(-0.4,1.3)
ax[0].set_ylim(-0.4,1.3)
ax[0].axhline(0, linewidth=2, color = 'k') 
ax[0].axvline(0, linewidth=2, color = 'k')
ax[0].plot(ax[0].get_xlim(), ax[0].get_ylim(), ls="--", c=".3")
# Feature importance
ax[1].bar(np.arange(rfr.n_features_),rfr.feature_importances_)
ax[1].set_xlabel('latent space representation ', fontsize=17)
ax[1].tick_params(axis='both', labelsize=16)
ax[1].set_title('feature importance', fontsize=20)

plt.savefig('plot_DMS_2H11.pdf')
